In [ ]:
import os

import math

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras


## image, label 설정
img_path = 'C:/cube_resized/1300/'
lbl_path = 'C:/merged_label/1300/'

str_path = next(os.walk(img_path))[1]

cnt = 0
files = []
for i in str_path:
    for j in next(os.walk(lbl_path + i))[2]:
        files.append(i + '/' + j[-8:])
        cnt += 1

low_image = np.zeros((cnt,256,256,3))
low_label = np.zeros((cnt,256,256))

for n, strength in enumerate(str_path):
    image_paths = img_path + strength + '/'
    label_paths = lbl_path + strength + '/'
    ep = len(next(os.walk(label_paths))[2])
    for i, lbl in enumerate(next(os.walk(label_paths))[2]):
        
        img = lbl[-8:]
        
        image = plt.imread(image_paths + img)
        low_image[n*ep + i] = image[:,:,:3]
        
        label = plt.imread(label_paths + lbl)
        if label.ndim == 3:
            low_label[n*ep + i] = label[:,:,:1].reshape(256,256)
        else:
            low_label[n*ep + i] = label
            
            
# ## 시각화
# plt.figure(figsize=(15,15))
# plt.subplot(2,2,1)
# plt.imshow(low_image[5])
# plt.subplot(2,2,2)
# plt.imshow(low_label[5])
# plt.subplot(2,2,3)
# plt.imshow(low_image[15])
# plt.subplot(2,2,4)
# plt.imshow(low_label[15])


# Kfold
from sklearn.model_selection import KFold

df = pd.DataFrame({'file_name':files,'fold':[-1 for i in range(len(low_image))]})

kf = KFold(n_splits=5, shuffle=True, random_state=32)

for idx, (t,v) in enumerate(kf.split(low_image)):
    df.loc[v, 'fold'] = idx


# Augmentation
import albumentations as A

class Augmentation:
    def __init__(self, size, mode='train'):
        if mode == 'train':
            self.transform = A.Compose([
                A.HorizontalFlip(p=0.5),
                A.VerticalFlip(p=0.5),
                A.ShiftScaleRotate(
                    p=0.5,
                    shift_limit=0.05,
                    scale_limit=0.05,
                    rotate_limit=45,
                ),
                A.RandomRotate90(p=0.5),
                A.RandomBrightnessContrast(p=0.2),
            ])
    def __call__(self, **kwargs):
        if self.transform:
            augmented = self.transform(**kwargs)
            img = augmented['image']
            mask = augmented['mask']
            return img, mask
        
        
# Generator
class DataGenerator(keras.utils.Sequence):
    def __init__(self, batch_size, df, image, label,
                fold, image_size, mode='train', shuffle=True):
        self.batch_size = batch_size
        self.fold = fold
        self.image_size = image_size
        self.mode = mode
        self.shuffle = shuffle
        
        self.image = image
        self.label = label
        
        self.df = df
        
        # train : 해당 fold에 해당하는 데이터를 제외하고 가져옴
        # val : 해당 fold에 해당하는 데이터를 가져옴
        if self.mode == 'train':
            self.df = self.df[self.df['fold'] != self.fold]
        elif self.mode == 'val':
            self.df = self.df[self.df['fold'] == self.fold]
        
        
        # Data Augmentation
        self.transform = Augmentation(image_size, mode)
        
        # 학습을 시작할 때도(첫 epoch) 셔플해주려면,
        self.on_epoch_end()
        
    def __len__(self):
        return math.ceil(len(self.df) / self.batch_size)
        
    def __getitem__(self, idx):
        
        strt = idx * self.batch_size
        fin = (idx + 1) * self.batch_size
        data = self.df.iloc[strt:fin]
        
        batch_x, batch_y = self.image, self.label
        
        return np.array(batch_x), np.array(batch_y)
        
    
    def on_epoch_end(self):
        
        if self.shuffle:
            self.df = self.df.sample(frac=1).reset_index(drop=True)

train_generator = DataGenerator(
    batch_size=8,
    df=df,
    image=low_image,
    label=low_label,
    fold=1,
    image_size=256,
    mode='train',
    shuffle=True)

valid_generator = DataGenerator(
    batch_size=8,
    df=df,
    image=low_image,
    label=low_label,
    fold=1,
    image_size=256,
    mode='val',
    shuffle=True)


# Loss
import tensorflow.keras.losses as losses

def iou(y_true, y_pred):
    
    smooth = 1.  # 0으로 나누는 것 방지
    
    # Flatten
    y_true = tf.reshape(y_true, [-1])
    y_pred = tf.reshape(y_pred, [-1])
    
    intersection = tf.reduce_sum(y_true * y_pred)
    union = tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) - intersection
    
    score = intersection / (union + smooth)
    return score

def dice_coef(y_true, y_pred):
    
    smooth = 1.  # 0으로 나누는 것 방지
    
    # Flatten
    y_true = tf.reshape(y_true, [-1])
    y_pred = tf.reshape(y_pred, [-1])
    
    intersection = tf.reduce_sum(y_true * y_pred)
    
    score = (2 * intersection) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth)
    return score

def dice_loss(y_true, y_pred):
    
    loss = 1 - dice_coef(y_true, y_pred)

    return loss

def bce_dice_loss(y_true, y_pred):
    
    loss = 1. * losses.binary_crossentropy(y_true,y_pred) + 1. * dice_loss(y_true, y_pred)

    return loss

def Predict_model(images):
    
    image_datas = np.zeros((len(images), 256,256,3))
    
    for i in range(len(images)):
        image_datas[i] = images[i][:,:,:3]
        
    preds = model.predict(image_datas)
    
    for i in range(len(images)):
        preds[i] = preds[i]
        
    return preds


# model compile
from keras_unet.models import custom_unet
from keras.callbacks import EarlyStopping

result = []


model = custom_unet(
    input_shape=(256, 256, 3),
    use_batch_norm=True,  # True로 해주는게 좋음
    upsample_mode='deconv',  # deconv가 일반적
    use_attention=True,  # hyperparameter로 비교하지만, 사용하는게 좋음
    filters=17,
    num_layers=3 ,  # conv layer갯수로 얼마나 깊은지 hyperparameter
    num_classes=4,
    dropout_type='spatial',  # hyperparameter의 의미로 'standard'와 비교해서 쓰지만, 일반적으로 spatial로 하는게 성능이 좋다고 알려짐
    dropout=0.5,
    output_activation='sigmoid')

model.compile(optimizer='adam', loss=bce_dice_loss, metrics=['accuracy', iou])


early_stopping = EarlyStopping(monitor = 'val_accracy', min_delta = 0, patience = 20, mode = 'auto')

# 학습
history = model.fit(
    train_generator,
    validation_data=valid_generator,
    epochs=1000,
    verbose=1,
    callbacks = [early_stopping])










## image, label 설정
test_img_path_24 = 'C:/cube_resized/1300/24/'

num_24 = len(next(os.walk(test_img_path_24))[2])

test_images_24 = np.zeros((num_24,256,256,3))

for i in range(num_24):
    test_images_24[i] = plt.imread(test_img_path_24 + next(os.walk(test_img_path_24))[2][i])[:,:,:3]
    
    
test_labels_24 = Predict_model(test_images_24)



porosity_24 = []
for i in range(test_labels_24.shape[0]):
    cnt = 0
    for j in range(256):
        for k in range(256):
            cnt += test_labels_24[i][j][k]
                
    porosity_24.append(cnt/(256*256)*100)
    
    





## image, label 설정
test_img_path_35 = 'C:/cube_resized/1300/35/'

num_35 = len(next(os.walk(test_img_path_35))[2])

test_images_35 = np.zeros((num_35,256,256,3))

for i in range(num_35):
    test_images_35[i] = plt.imread(test_img_path_35 + next(os.walk(test_img_path_35))[2][i])[:,:,:3]
    
    
test_labels_35 = Predict_model(test_images_35)



porosity_35 = []
for i in range(test_labels_35.shape[0]):
    cnt = 0
    for j in range(256):
        for k in range(256):
            cnt += test_labels_35[i][j][k]
                
    porosity_35.append(cnt/(256*256)*100)
    
    



## image, label 설정
test_img_path_50 = 'C:/cube_resized/1300/50/'

num_50 = len(next(os.walk(test_img_path_50))[2])

test_images_50 = np.zeros((num_50,256,256,3))

for i in range(num_50):
    test_images_50[i] = plt.imread(test_img_path_50 + next(os.walk(test_img_path_50))[2][i])[:,:,:3]
    
    
test_labels_50 = Predict_model(test_images_50)



porosity_50 = []
for i in range(test_labels_50.shape[0]):
    cnt = 0
    for j in range(256):
        for k in range(256):
            cnt += test_labels_50[i][j][k]
                
    porosity_50.append(cnt/(256*256)*100)
    
    



for i in range(len(porosity_24)):
    porosity_24[i] = porosity_24[i][0]
    
for i in range(len(porosity_35)):
    porosity_35[i] = porosity_35[i][0]
    
for i in range(len(porosity_50)):
    porosity_50[i] = porosity_50[i][0]



avg_pore_24 = np.average(porosity_24)
avg_pore_35 = np.average(porosity_35)
avg_pore_50 = np.average(porosity_50)


        
        
        
from keras.models import load_model

model.save('multi_first.h5')

In [6]:
a = 'sp0001.png'

a[-8:-4]

'0001.'